# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [12]:
import json
import pandas as pd
import pickle

with open('C:\\Users\\sankl\\OneDrive\\Документы\\contributors_sample.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
a = []
for i in range(len(data)):
    a.append(data[i])
table = pd.DataFrame(a)
table1 = table[:3]
table1

,username,name,sex,address,mail,jobs,id
0,uhebert,Lindsey Nguyen,F,"01261 Cameron Spring\nTaylorfurt, AK 97791",jsalazar@gmail.com,"[Energy engineer, Engineer, site, Environmenta...",35193
1,vickitaylor,Cheryl Lewis,F,"66992 Welch Brooks\nMarshallshire, ID 56004",bhudson@gmail.com,"[Music therapist, Volunteer coordinator, Desig...",91970
2,sheilaadams,Julia Allen,F,Unit 1632 Box 2971\nDPO AE 23297,darren44@yahoo.com,"[Management consultant, Engineer, structural, ...",1848091


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [13]:
table2 = list(table['mail'])
a1 = []
for i in table2:
    k = i[i.find('@'):]
    if k not in a1:
        a1.append(k)
a1

['@gmail.com', '@yahoo.com', '@hotmail.com']

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [14]:
username = str(input(''))
def find_user(username):
    k = table[table['username'] == username]
    if not k.empty:
        print(k)
    else:
        raise ValueError
find_user(username)

uhebert
  username            name sex                                     address  \
0  uhebert  Lindsey Nguyen   F  01261 Cameron Spring\nTaylorfurt, AK 97791   

                 mail                                               jobs  \
0  jsalazar@gmail.com  [Energy engineer, Engineer, site, Environmenta...   

      id  
0  35193  


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [17]:
count_all = list(table['sex'])
count_women = count_all.count('F')
count_men = count_all.count('M') 
print(f'Количество женщин: {count_women} \nКоличество мужчин: {count_men}')

Количество женщин: 2136 
Количество мужчин: 2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [18]:
contributors = table[['id','username','sex']].copy()
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
new_recipes = pd.merge(recipes, contributors, left_on='contributor_id', right_on='id', how='left').drop(['id_y'], axis=1)
len(new_recipes[new_recipes['username'].isnull()])

### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [20]:
d = {}
for user in data:
    for job in user['jobs']:
        if job not in d:
            d[job] = [user['username']]
        else:
            d[job] = [*d[job], user['username']]


2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
with open('job_people.pickle', 'wb') as f:
    pickle.dump(d, f)
    
with open('job_people.json', 'w') as f: 
    json.dump(d, f, indent=2)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.pickle', 'rb') as f:
    d23 = pickle.load(f)

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
from bs4 import BeautifulSoup
with open('steps_sample.xml') as f:
    ab = BeautifulSoup(f, 'xml')
d1 = {}
for recipe in ab.find_all("recipe"):
    id1 = recipe.find('id').text
    steps = [step.text for step in recipe.find('steps').find_all('step')]
    d1[id1] = steps



3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
d2 = {}
for recipe in ab.find_all("recipe"):
    steps = recipe.find_all("step")
    count_step = len(steps)
    if count_step in d2:
        d2[count_step].append(recipe.id.string)
    else:
        d2[count_step] = [recipe.id.string]


3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
recipes3 = []
for recipe in ab.find_all("recipe"):
    for step in recipe.steps.find_all('step'):
        if (str(step).find('has_hours') != -1) or (str(step).find('has_minutes') != -1):
            recipes3.append(recipe.id.string)
            break


3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
d4 = []
recipes = pd.read_csv("recipes_sample.csv", sep=',')
for recipe in ab.find_all("recipe"):
    l = {'id': int(recipe.find('id').text), 'n_steps': len(recipe.find_all('step'))}
    d4.append(l)
table4 = pd.DataFrame(d34)
recipes=pd.read_csv('recipes_sample.csv')
recipes=pd.DataFrame(recipes)
recipes2 = pd.merge(recipes, table34, how='left', on='id')
recipes2['n_steps']=recipes2['n_steps_y']
recipes2 = recipes2.drop('n_steps_x',axis=1)
recipes2 = recipes2.drop('n_steps_y',axis=1)
recipes2

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
if recipes2['n_steps'].isna().sum() == 0:
    print('Не содержит пропуски')
    recipes2['n_steps'] = recipes2['n_steps'].astype(int)
    recipes2.to_csv('recipes_sample_with_filled_nsteps.csv', encoding='utf-8')